# 简单RAG技术实现详解

## 什么是RAG？给小朋友的解释

想象一下，你在写作文时需要查找一些资料。通常你会怎么做？可能会去查书、上网搜索，找到相关内容后，再根据这些内容写出你自己的文章，对吧？

**RAG（检索增强生成）**就像是一个非常聪明的小助手，它可以：
1. 先帮你在很多书本中**查找**（检索）相关内容
2. 然后根据找到的内容**创作**（生成）出回答

就像你做数学题时，如果不记得公式，你会先翻课本找到公式，再用这个公式解题。RAG也是这样工作的！

## 为什么需要RAG？

AI模型（如ChatGPT）虽然很聪明，但它们可能：
- 知识有限或过时（只学习到训练时的知识）
- 有时会
答案（因为没有查证事实）

RAG通过让AI先查阅资料，再回答问题，可以：
- 提供最新、准确的信息
- 减少错误和编造
- 给出有根据的回答

## 本实验将展示基础RAG的工作流程

我们将一步步实现：
1. 读取PDF文件中的文本
2. 把长文本分成小片段
3. 将文本转换成电脑能理解的数字（嵌入向量）
4. 找出与问题最相关的文本片段
5. 让AI根据这些相关片段生成回答
6. 评估回答的质量

跟着代码一起动手，你也能拥有自己的AI助手！

---

# Introduction to Simple RAG

Retrieval-Augmented Generation (RAG) is a hybrid approach that combines information retrieval with generative models. It enhances the performance of language models by incorporating external knowledge, which improves accuracy and factual correctness.

In a Simple RAG setup, we follow these steps:

1. **Data Ingestion**: Load and preprocess the text data.
2. **Chunking**: Break the data into smaller chunks to improve retrieval performance.
3. **Embedding Creation**: Convert the text chunks into numerical representations using an embedding model.
4. **Semantic Search**: Retrieve relevant chunks based on a user query.
5. **Response Generation**: Use a language model to generate a response based on retrieved text.

This notebook implements a Simple RAG approach, evaluates the model’s response, and explores various improvements.

## Setting Up the Environment
We begin by importing necessary libraries.

In [ ]:
# 导入所需的库
import fitz  # PyMuPDF库，用于读取PDF文件
import os  # 操作系统相关功能，如读取环境变量
import numpy as np  # 数学计算库，处理向量和矩阵
import json  # 处理JSON数据格式
from openai import OpenAI  # 调用OpenAI的API接口

## 从PDF文件中提取文本

首先，我们需要从PDF文件中获取文本内容。想象一下，我们要阅读一本电子书，但计算机需要先
这本书，才能理解其中的内容。

## Extracting Text from a PDF File
To implement RAG, we first need a source of textual data. In this case, we extract text from a PDF file using the PyMuPDF library.

In [ ]:
def extract_text_from_pdf(pdf_path):
    """
    从PDF文件中提取文本内容。
    
    想象成：把一本电子书的每一页内容都读出来，然后合并成一个长文本。
    
    参数:
    pdf_path (str): PDF文件的路径，类似于图书的位置。
    
    返回:
    str: 从PDF中提取的所有文本内容。
    """
    # 打开PDF文件，就像打开一本书
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 创建一个空字符串，用来存储所有页面的文本

    # 遍历PDF的每一页，就像一页一页地翻书
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 获取当前页
        text = page.get_text("text")  # 提取当前页的文本
        all_text += text  # 将当前页的文本添加到总文本中

    return all_text  # 返回提取的所有文本

## 将文本分成小块

想象一下，如果一本书有500页，一次性阅读可能很困难。所以我们把它分成小章节，每次只读一小部分。这就是
的概念。

为什么我们需要重叠的块？想象你正在阅读一个故事，如果我们把书分成完全不重叠的章节，一个重要的想法可能会被切分在两个章节之间。通过让章节有一点重叠，我们可以确保不会丢失这些重要的连接。

## Chunking the Extracted Text
Once we have the extracted text, we divide it into smaller, overlapping chunks to improve retrieval accuracy.

In [ ]:
def chunk_text(text, n, overlap):
    """
    将长文本分成多个小块，并且这些小块之间有一定的重叠。
    
    想象一下：
    如果我们有一段文字
，
    使用n=6，overlap=2进行分块，我们会得到：
    1. 

    2. 

    3. 

    4. 

    5. 

    
    参数:
    text (str): 要分块的文本。
    n (int): 每个块的大小（字符数）。
    overlap (int): 相邻块之间重叠的字符数。
    
    返回:
    List[str]: 文本块列表。
    """
    chunks = []  # 创建一个空列表，用于存储分块后的文本
    
    # 以(n - overlap)的步长遍历文本
    # 例如，如果块大小为100，重叠为20，则每次前进80个字符
    for i in range(0, len(text), n - overlap):
        # 从位置i开始，取n个字符，添加到chunks列表中
        chunks.append(text[i:i + n])

    return chunks  # 返回所有文本块的列表

## 设置OpenAI API客户端

我们需要初始化OpenAI的客户端，这就像设置一个特殊的电话，让我们能够和AI模型进行通信。

## Setting Up the OpenAI API Client
We initialize the OpenAI client to generate embeddings and responses.

In [ ]:
# 初始化OpenAI客户端，设置API的网址和密钥
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",  # API的网址
    api_key=os.getenv("OPENAI_API_KEY")  # 从环境变量中获取API密钥
)

## 从PDF文件中提取并分块文本

接下来，我们要加载PDF文件，提取文本，并将其分成小块。就像我们把一本书拆分成多个小章节。

## Extracting and Chunking Text from a PDF File
Now, we load the PDF, extract text, and split it into chunks.

In [ ]:
# 定义PDF文件的路径
pdf_path = "data/AI_information.pdf"

# 从PDF文件中提取文本
extracted_text = extract_text_from_pdf(pdf_path)

# 将提取的文本分成1000个字符的小块，每块之间有200个字符的重叠
# 这就像把一本长书分成小章节，而且章节之间有一些重叠内容
text_chunks = chunk_text(extracted_text, 1000, 200)

# 打印创建的文本块数量
print("文本块数量:", len(text_chunks))

# 打印第一个文本块内容
print("\n第一个文本块:")
print(text_chunks[0])

## 为文本块创建嵌入向量

**什么是嵌入向量？**

想象一下，我们需要把文本转换成电脑能理解的形式。嵌入向量就是一组数字，可以捕捉文本的含义。

例如：
- 
 可能转换为 [0.2, 0.8, 0.1, ...]
- 
 可能转换为 [0.2, 0.7, 0.2, ...]
- 
 可能转换为 [0.8, 0.1, 0.7, ...]

因为
和
意思相近，所以它们的嵌入向量也会很相似。而
的含义完全不同，所以它的嵌入向量会与前两个差别很大。

这样，电脑就能通过比较向量来判断文本的相似度！

## Creating Embeddings for Text Chunks
Embeddings transform text into numerical vectors, which allow for efficient similarity search.

In [ ]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    为给定的文本创建嵌入向量，使用指定的模型。
    
    嵌入向量是什么？想象一下把文字变成一长串数字，
    这些数字保留了文字的意思，让电脑能理解和比较不同文本的相似度。
    
    参数:
    text (str): 需要创建嵌入向量的输入文本。
    model (str): 用于创建嵌入向量的模型。默认是"BAAI/bge-en-icl"。
    
    返回:
    dict: 包含嵌入向量的API响应。
    """
    # 使用指定的模型为输入文本创建嵌入向量
    response = client.embeddings.create(
        model=model,  # 使用哪个模型来创建嵌入向量
        input=text    # 需要转换的文本
    )
    
    return response  # 返回包含嵌入向量的响应

# 为所有文本块创建嵌入向量
response = create_embeddings(text_chunks)

## 执行语义搜索

**什么是语义搜索？**

语义搜索是找出与我们问题意思最接近的文本块。就像你问老师一个问题，老师从课本中找出最相关的部分来回答你。

首先，我们需要一个方法来比较两段文本的相似度。这里我们使用**余弦相似度**——一种数学方法，用来比较两个向量的方向是否相似。

## Performing Semantic Search
We implement cosine similarity to find the most relevant text chunks for a user query.

In [ ]:
def cosine_similarity(vec1, vec2):
    """
    计算两个向量之间的余弦相似度。
    
    余弦相似度是什么？想象两支铅笔从同一个点出发，指向不同方向。
    - 如果两支铅笔指向完全相同的方向，它们的相似度是1
    - 如果它们指向完全相反的方向，相似度是-1
    - 如果它们互相垂直，相似度是0
    
    参数:
    vec1 (np.ndarray): 第一个向量（比如问题的嵌入向量）。
    vec2 (np.ndarray): 第二个向量（比如文本块的嵌入向量）。
    
    返回:
    float: 两个向量之间的余弦相似度，范围在-1到1之间。
    """
    # 计算两个向量的点积，然后除以它们的范数的乘积
    # 点积：两个向量对应位置的值相乘后求和
    # 范数：向量的长度或大小
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    使用给定的查询和嵌入向量对文本块进行语义搜索。
    
    这就像在图书馆里：
    1. 你有一个问题（查询）
    2. 图书管理员（搜索函数）帮你在所有书（文本块）中找到最相关的几页
    
    参数:
    query (str): 用于语义搜索的查询问题。
    text_chunks (List[str]): 要搜索的文本块列表。
    embeddings (List[dict]): 文本块的嵌入向量列表。
    k (int): 要返回的最相关文本块的数量。默认为5。
    
    返回:
    List[str]: 基于查询的前k个最相关文本块的列表。
    """
    # 为查询创建嵌入向量
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []  # 初始化一个列表来存储相似度分数
    
    # 计算查询嵌入向量与每个文本块嵌入向量之间的相似度分数
    for i, chunk_embedding in enumerate(embeddings):
        # 使用余弦相似度计算查询与当前文本块的相似度
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        similarity_scores.append((i, similarity_score))  # 将索引和相似度分数添加到列表中
    
    # 按相似度降序排序（最相似的在前面）
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    # 获取前k个最相似文本块的索引
    top_indices = [index for index, _ in similarity_scores[:k]]
    # 返回前k个最相关的文本块
    return [text_chunks[index] for index in top_indices]


## 在提取的文本块上运行查询

现在我们要用一个实际问题来测试我们的系统。我们从一个JSON文件中加载测试问题，然后使用语义搜索找出最相关的文本块。

## Running a Query on Extracted Chunks

In [ ]:
# 从JSON文件加载验证数据
with open('data/val.json') as f:
    data = json.load(f)  # 读取JSON文件内容

# 提取验证数据中的第一个问题
query = data[0]['question']  # 获取第一个问题

# 执行语义搜索，找出与查询最相关的前2个文本块
top_chunks = semantic_search(query, text_chunks, response.data, k=2)

# 打印查询问题
print("问题:", query)

# 打印找到的最相关的两个文本块
for i, chunk in enumerate(top_chunks):
    print(f"上下文 {i + 1}:\n{chunk}\n======================================")

## 基于检索到的文本块生成回答

现在我们有了与问题最相关的文本块，接下来要让AI模型基于这些文本生成一个回答。

这就像：
1. 你把找到的相关书页（上下文）给了一个非常聪明的朋友
2. 告诉这位朋友你的问题
3. 你的朋友只用这些书页上的信息来回答你的问题

## Generating a Response Based on Retrieved Chunks

In [ ]:
# 定义AI助手的系统提示
system_prompt = "你是一个AI助手，严格基于给定的上下文回答问题。如果答案不能直接从提供的上下文中得出，请回复：'我没有足够的信息来回答这个问题。'"

def generate_response(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    基于系统提示和用户消息从AI模型生成回答。
    
    这就像告诉AI两件事：
    1. 它应该如何表现（系统提示）
    2. 用户的问题是什么（用户消息）
    
    参数:
    system_prompt (str): 引导AI行为的系统提示。
    user_message (str): 用户的问题或消息。
    model (str): 用于生成回答的模型。默认是"meta-llama/Llama-3.2-3B-Instruct"。
    
    返回:
    dict: AI模型的响应。
    """
    # 创建聊天完成请求
    response = client.chat.completions.create(
        model=model,  # 使用哪个AI模型
        temperature=0,  # 温度为0表示尽可能确定性的回答，减少随机性
        messages=[  # 消息列表，包含系统提示和用户消息
            {"role": "system", "content": system_prompt},  # 系统角色的消息
            {"role": "user", "content": user_message}  # 用户角色的消息
        ]
    )
    return response  # 返回AI的响应

# 基于最相关的文本块创建用户提示
# 将每个文本块格式化为上下文，然后用换行符连接起来
user_prompt = "\n".join([f"上下文 {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
# 在上下文后面添加问题
user_prompt = f"{user_prompt}\n问题: {query}"

# 生成AI回答
ai_response = generate_response(system_prompt, user_prompt)

## 评估AI的回答

最后一步是检查我们的AI回答质量如何。我们将它与预期的
回答进行比较，并给出一个分数。

这就像老师批改作业：
- 完全正确得1分
- 部分正确得0.5分
- 错误得0分

## Evaluating the AI Response
We compare the AI response with the expected answer and assign a score.

In [ ]:
# 定义评估系统的系统提示
evaluate_system_prompt = "你是一个智能评估系统，负责评估AI助手的回答。如果AI助手的回答与真实回答非常接近，给予1分。如果回答相对于真实回答不正确或不令人满意，给予0分。如果回答部分符合真实回答，给予0.5分。"

# 创建评估提示，结合用户查询、AI回答和真实回答
evaluation_prompt = f"用户问题: {query}\nAI回答:\n{ai_response.choices[0].message.content}\n真实回答: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# 使用评估系统提示和评估提示生成评估响应
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 打印评估响应
print(evaluation_response.choices[0].message.content)

In [11]:
# Define the system prompt for the evaluation system
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# Create the evaluation prompt by combining the user query, AI response, true response, and evaluation system prompt
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# Generate the evaluation response using the evaluation system prompt and evaluation prompt
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# Print the evaluation response
print(evaluation_response.choices[0].message.content)

Based on the evaluation criteria, I would assign a score of 0.8 to the AI assistant's response.

The AI assistant's response is very close to the true response, but there are some minor differences. The true response mentions "transparency" and "accountability" explicitly, which are not mentioned in the AI assistant's response. However, the overall meaning and content of the response are identical, and the AI assistant's response effectively conveys the importance of Explainable AI in building trust and ensuring fairness in AI systems.

Therefore, the score of 0.8 reflects the AI assistant's response being very close to the true response, but not perfectly aligned.
